# Process and Plot the Peak Historical Loads by Balancing Authority

This notebook analyzes the historical time series of extreme loads (summer and winter) by BA.

In [ ]:
# Start by importing the packages we need:
import os

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from glob import glob
from matplotlib import pyplot
from mpl_toolkits.axes_grid1 import make_axes_locatable


## Set the Directory Structure

In [ ]:
# Identify the data input and output directories:
load_data_input_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/outputs/tell_output/historic/'
temp_data_input_dir =  '/Volumes/LaCie/Big_Data/wrf_to_tell/wrf_tell_counties_output/historic/'
metadata_input_dir =  '/Users/burl878/Documents/IMMM/Data/TELL_Input_Data/tell_raw_data/County_Shapefiles/'
data_output_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/outputs/postprocessed/ba_load_time_series/'
image_output_dir =  '/Users/burl878/Documents/IMMM/Images/TELL/Analysis/BA_Peak_Loads/'


## Set the BA to Process and Plot

In [ ]:
ba_to_process = 'BPAT'


## Process the Historical Peak Loads

In [ ]:
# Define a function to process the time series of historical peak loads:
def process_ba_historical_peak_loads(ba_to_process: str, load_data_input_dir: str, data_output_dir: str):
    
    #Initiate a counter and empty dataframe to store the results:
    counter = 0;
    stats_df = pd.DataFrame()
    
    # Loop over the years and find the maximum summer and winter load in each year:
    for year in range(1980,2020,1):
        # Iterate the counter by one:
        counter = counter + 1 

        # Read in the historical interconnection loads file:
        ba_df = pd.read_csv((load_data_input_dir + '/' + str(year) + '/' + 'TELL_Balancing_Authority_Hourly_Load_Data_' + str(year) + '_Scaled_' + str(year) + '.csv'))
    
        # Subset to just the data for the BA being processed:
        ba_df = ba_df.loc[ba_df['BA_Code'] == ba_to_process]
    
        # Set the time value as a datetime variable:
        ba_df['Datetime'] = pd.to_datetime(ba_df['Time_UTC'])
        
        # Add columns with the year and month values to be used in grouping:
        ba_df['Year'] = ba_df['Datetime'].dt.strftime('%Y').astype(str).astype(int)
        ba_df['Month'] = ba_df['Datetime'].dt.strftime('%m').astype(str).astype(int)
    
        # Compute the annual mean and standard deviation of the load:
        ba_df['Year_Mean_Load'] = ba_df.groupby('Year')['Scaled_TELL_BA_Load_MWh'].transform('mean').round(2)
        ba_df['Year_STD_Load'] = ba_df.groupby('Year')['Scaled_TELL_BA_Load_MWh'].transform('std').round(2)
    
        # Compute the hourly normalized load by subtracting the annual mean and dividing by the annual standard deviation:
        ba_df['Normalized_Load'] = ((ba_df['Scaled_TELL_BA_Load_MWh'] - ba_df['Year_Mean_Load']) / ba_df['Year_STD_Load']).round(2)
    
        # Subset to just the columns we need:
        ba_df = ba_df[['Time_UTC', 'Year', 'Month', 'Scaled_TELL_BA_Load_MWh', 'Normalized_Load', 'Year_Mean_Load']].copy()
    
        # Subset the data to just the summer and winter months:
        winter_df = ba_df.loc[(ba_df['Month'] == 1) | (ba_df['Month'] == 2) | (ba_df['Month'] == 3) | (ba_df['Month'] == 10) | (ba_df['Month'] == 11) | (ba_df['Month'] == 12)]
        summer_df = ba_df.loc[(ba_df['Month'] == 4) | (ba_df['Month'] == 5) | (ba_df['Month'] == 6) | (ba_df['Month'] == 7) | (ba_df['Month'] == 8) | (ba_df['Month'] == 9)]
        
        # Find the row of the maximum load for each subset:
        all_max_index = ba_df['Scaled_TELL_BA_Load_MWh'].idxmax()
        winter_max_index = winter_df['Scaled_TELL_BA_Load_MWh'].idxmax()
        summer_max_index = summer_df['Scaled_TELL_BA_Load_MWh'].idxmax()
                
        # Put the statistics in a new dataframe:
        stats_df.loc[counter, 'Year'] = str(year)
        stats_df.loc[counter, 'Mean_Load_MWh'] = ba_df['Year_Mean_Load'].mean().round(2)
        stats_df.loc[counter, 'All_Max_Time'] = ba_df.loc[all_max_index, 'Time_UTC']
        stats_df.loc[counter, 'All_Max_Load_MWh'] = ba_df.loc[all_max_index, 'Scaled_TELL_BA_Load_MWh']
        stats_df.loc[counter, 'All_Max_Load_Norm'] = ba_df.loc[all_max_index, 'Normalized_Load']
        stats_df.loc[counter, 'Win_Max_Time'] = winter_df.loc[winter_max_index, 'Time_UTC']
        stats_df.loc[counter, 'Win_Max_Load_MWh'] = winter_df.loc[winter_max_index, 'Scaled_TELL_BA_Load_MWh']
        stats_df.loc[counter, 'Win_Max_Load_Norm'] = winter_df.loc[winter_max_index, 'Normalized_Load']
        stats_df.loc[counter, 'Sum_Max_Time'] = summer_df.loc[summer_max_index, 'Time_UTC']
        stats_df.loc[counter, 'Sum_Max_Load_MWh'] = summer_df.loc[summer_max_index, 'Scaled_TELL_BA_Load_MWh']
        stats_df.loc[counter, 'Sum_Max_Load_Norm'] = summer_df.loc[summer_max_index, 'Normalized_Load']

        # Clean up and move to the next year:
        del ba_df, winter_df, summer_df, all_max_index, winter_max_index, summer_max_index
        
    # Write out the time series dataframe to a .csv file:
    stats_df.to_csv((os.path.join(data_output_dir + ba_to_process + '_Peak_Load_Statistics.csv')), sep=',', index=False)
    
    return stats_df


In [ ]:
stats_df = process_ba_historical_peak_loads(ba_to_process = ba_to_process, 
                                            load_data_input_dir = load_data_input_dir, 
                                            data_output_dir = data_output_dir)

stats_df


In [ ]:
# Define a function to plot the time series of extreme loads by BA:
def plot_ba_historical_peak_loads(ba_to_process: str, data_output_dir: str, image_output_dir: str, image_resolution: int, save_images=False):
    
    # Check to see if the output already exist and if not then process it:
    if os.path.isfile((os.path.join(data_output_dir + ba_to_process + '_Peak_Load_Statistics.csv'))) == True:
       # Load in the pre-processed data:
       stats_df = pd.read_csv((os.path.join(data_output_dir + ba_to_process + '_Peak_Load_Statistics.csv')))
    else:
       stats_df = process_ba_historical_peak_loads(ba_to_process = ba_to_process, load_data_input_dir = load_data_input_dir, data_output_dir = data_output_dir)
    
    # Find the four highest values:
    winter_peak_raw = stats_df.nlargest(4, 'Win_Max_Load_MWh')
    winter_peak_norm = stats_df.nlargest(4, 'Win_Max_Load_Norm')
    summer_peak_raw = stats_df.nlargest(4, 'Sum_Max_Load_MWh')
    summer_peak_norm = stats_df.nlargest(4, 'Sum_Max_Load_Norm')
    
    # Concatenate and print the dates of the four highest winter and summer load dates:
    max_times = pd.concat([winter_peak_norm['Win_Max_Time'],summer_peak_norm['Sum_Max_Time']])
    
    # Make the plot:
    plt.figure(figsize=(24, 15))
    plt.rcParams['font.size'] = 16
    
    plt.subplot(211)
    plt.plot(stats_df['Year'], stats_df['Win_Max_Load_MWh'], color='b', linestyle='-', label='Winter Peak Load', linewidth=3)
    plt.plot(stats_df['Year'], stats_df['Sum_Max_Load_MWh'], color='r', linestyle='-', label='Summer Peak Load', linewidth=3)
    plt.scatter(winter_peak_raw['Year'], winter_peak_raw['Win_Max_Load_MWh'], s=100, c='blue')
    plt.scatter(summer_peak_raw['Year'], summer_peak_raw['Sum_Max_Load_MWh'], s=100, c='red')
    plt.xlim([1979, 2020]); 
    plt.grid(True)
    plt.legend(loc='upper left', prop={'size': 14})
    plt.ylabel('BA Load [MWh]')
    plt.title((ba_to_process + ' Peak Loads: 1980-2019'))
    
    plt.subplot(212)
    plt.plot(stats_df['Year'], stats_df['Win_Max_Load_Norm'], color='b', linestyle='-', label='Winter Peak Load', linewidth=3)
    plt.plot(stats_df['Year'], stats_df['Sum_Max_Load_Norm'], color='r', linestyle='-', label='Summer Peak Load', linewidth=3)
    plt.scatter(winter_peak_norm['Year'], winter_peak_norm['Win_Max_Load_Norm'], s=100, c='blue')
    plt.scatter(summer_peak_norm['Year'], summer_peak_norm['Sum_Max_Load_Norm'], s=100, c='red')
    plt.xlim([1979, 2020]); 
    plt.grid(True)
    plt.xlabel('Year');
    plt.ylabel('Annually Normalized BA Load')
    plt.title(('Normalized Peak Loads'))
    
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       filename = (ba_to_process + '_Peak_Load_Time_Series.png')
       plt.savefig(os.path.join(image_output_dir, filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')
    
    return max_times


In [ ]:
max_times = plot_ba_historical_peak_loads(ba_to_process = ba_to_process, 
                                          data_output_dir = data_output_dir,
                                          image_output_dir = image_output_dir, 
                                          image_resolution = 300, 
                                          save_images = True)

max_times


In [ ]:
# Define a function to plot the temperature during extreme load events:
def plot_peak_load_temperatures(ba_to_process: str, metadata_input_dir: str, temp_data_input_dir: str, data_output_dir: str, image_output_dir: str, image_resolution: int, save_images=False):

    # Read in the county shapefile and reassign the 'FIPS' variable as integers:
    counties_df = gpd.read_file(os.path.join(metadata_input_dir, r'tl_2020_us_county.shp')).rename(columns={'GEOID': 'FIPS'})
    counties_df['FIPS'] = counties_df['FIPS'].astype(int)
    
    if ba_to_process == 'BPAT':
       cold1_temp = pd.read_csv((os.path.join(temp_data_input_dir + '2004/' + '2004_01_05_17_UTC_County_Mean_Meteorology.csv')))
       cold1_temp['T2'] = (1.8 * (cold1_temp['T2'] - 273)) + 32
       cold1_label = 'BPAT Winter Peak One: 5-Jan 2004 1700 UTC'
       cold2_temp = pd.read_csv((os.path.join(temp_data_input_dir + '1989/' + '1989_02_02_17_UTC_County_Mean_Meteorology.csv')))
       cold2_temp['T2'] = (1.8 * (cold2_temp['T2'] - 273)) + 32
       cold2_label = 'BPAT Winter Peak Two: 2-Feb 1989 1700 UTC'
       cold3_temp = pd.read_csv((os.path.join(temp_data_input_dir + '1983/' + '1983_12_23_17_UTC_County_Mean_Meteorology.csv')))
       cold3_temp['T2'] = (1.8 * (cold3_temp['T2'] - 273)) + 32
       cold3_label = 'BPAT Winter Peak Three: 23-Dec 1983 1700 UTC'
       cold4_temp = pd.read_csv((os.path.join(temp_data_input_dir + '1982/' + '1982_01_06_17_UTC_County_Mean_Meteorology.csv')))
       cold4_temp['T2'] = (1.8 * (cold4_temp['T2'] - 273)) + 32
       cold4_label = 'BPAT Winter Peak Four: 6-Jan 1982 1700 UTC'
    
       warm1_temp = pd.read_csv((os.path.join(temp_data_input_dir + '1998/' + '1998_07_28_02_UTC_County_Mean_Meteorology.csv')))
       warm1_temp['T2'] = (1.8 * (warm1_temp['T2'] - 273)) + 32
       warm1_label = 'BPAT Summer Peak One: 28-Jul 1998 0200 UTC'
       warm2_temp = pd.read_csv((os.path.join(temp_data_input_dir + '2006/' + '2006_07_24_01_UTC_County_Mean_Meteorology.csv')))
       warm2_temp['T2'] = (1.8 * (warm2_temp['T2'] - 273)) + 32
       warm2_label = 'BPAT Summer Peak Two: 24-Jul 2006 0100 UTC'
       warm3_temp = pd.read_csv((os.path.join(temp_data_input_dir + '1981/' + '1981_08_10_02_UTC_County_Mean_Meteorology.csv')))
       warm3_temp['T2'] = (1.8 * (warm3_temp['T2'] - 273)) + 32
       warm3_label = 'BPAT Summer Peak Three: 10-Aug 1981 0200 UTC'
       warm4_temp = pd.read_csv((os.path.join(temp_data_input_dir + '2009/' + '2009_07_29_02_UTC_County_Mean_Meteorology.csv')))
       warm4_temp['T2'] = (1.8 * (warm4_temp['T2'] - 273)) + 32
       warm4_label = 'BPAT Summer Peak Four: 29-Jul 2009 0200 UTC'
    
    # Merge the ba_mapping_df and counties_df together using county FIPS codes to join them:
    cold1_df = counties_df.merge(cold1_temp, on='FIPS', how='left')
    cold2_df = counties_df.merge(cold2_temp, on='FIPS', how='left')
    cold3_df = counties_df.merge(cold3_temp, on='FIPS', how='left')
    cold4_df = counties_df.merge(cold4_temp, on='FIPS', how='left')
    warm1_df = counties_df.merge(warm1_temp, on='FIPS', how='left')
    warm2_df = counties_df.merge(warm2_temp, on='FIPS', how='left')
    warm3_df = counties_df.merge(warm3_temp, on='FIPS', how='left')
    warm4_df = counties_df.merge(warm4_temp, on='FIPS', how='left')
    
    # Create the cold event figure:
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(25,10))
    ax1 = cold1_df.plot(ax=axes[0, 0], column='T2', cmap='RdYlBu_r', vmin=-45, vmax=75, edgecolor='grey', linewidth=0.2, legend=True, legend_kwds={'label': ('Temperature ($^\circ$F)'), 'orientation': 'vertical'})
    ax1.set_title(cold1_label)
    ax2 = cold2_df.plot(ax=axes[0, 1], column='T2', cmap='RdYlBu_r', vmin=-45, vmax=75, edgecolor='grey', linewidth=0.2, legend=True, legend_kwds={'label': ('Temperature ($^\circ$F)'), 'orientation': 'vertical'})
    ax2.set_title(cold2_label)
    ax3 = cold3_df.plot(ax=axes[1, 0], column='T2', cmap='RdYlBu_r', vmin=-45, vmax=75, edgecolor='grey', linewidth=0.2, legend=True, legend_kwds={'label': ('Temperature ($^\circ$F)'), 'orientation': 'vertical'})
    ax3.set_title(cold3_label)
    ax4 = cold4_df.plot(ax=axes[1, 1], column='T2', cmap='RdYlBu_r', vmin=-45, vmax=75, edgecolor='grey', linewidth=0.2, legend=True, legend_kwds={'label': ('Temperature ($^\circ$F)'), 'orientation': 'vertical'})
    ax4.set_title(cold4_label)
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       filename = (ba_to_process + '_Winter_Peak_Maps.png')
       plt.savefig(os.path.join(image_output_dir, filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')
    
    # Create the hot event figure:
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(25,10))
    ax1 = warm1_df.plot(ax=axes[0, 0], column='T2', cmap='RdYlBu_r', vmin=45, vmax=115, edgecolor='grey', linewidth=0.2, legend=True, legend_kwds={'label': ('Temperature ($^\circ$F)'), 'orientation': 'vertical'})
    ax1.set_title(warm1_label)
    ax2 = warm2_df.plot(ax=axes[0, 1], column='T2', cmap='RdYlBu_r', vmin=45, vmax=115, edgecolor='grey', linewidth=0.2, legend=True, legend_kwds={'label': ('Temperature ($^\circ$F)'), 'orientation': 'vertical'})
    ax2.set_title(warm2_label)
    ax3 = warm3_df.plot(ax=axes[1, 0], column='T2', cmap='RdYlBu_r', vmin=45, vmax=115, edgecolor='grey', linewidth=0.2, legend=True, legend_kwds={'label': ('Temperature ($^\circ$F)'), 'orientation': 'vertical'})
    ax3.set_title(warm3_label)
    ax4 = warm4_df.plot(ax=axes[1, 1], column='T2', cmap='RdYlBu_r', vmin=45, vmax=115, edgecolor='grey', linewidth=0.2, legend=True, legend_kwds={'label': ('Temperature ($^\circ$F)'), 'orientation': 'vertical'})
    ax4.set_title(warm4_label)
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       filename = (ba_to_process + '_Summer_Peak_Maps.png')
       plt.savefig(os.path.join(image_output_dir, filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')


In [ ]:
plot_peak_load_temperatures(ba_to_process = ba_to_process,
                            metadata_input_dir = metadata_input_dir,
                            temp_data_input_dir = temp_data_input_dir,
                            data_output_dir = data_output_dir,
                            image_output_dir = image_output_dir, 
                            image_resolution = 300, 
                            save_images = True)
